# Note
### This notebook currently in progress and still an experiment
### Example output available in version 2

In [ ]:
save_loc = "/kaggle/tmp/model"

In [ ]:
!mkdir /kaggle/tmp
!mkdir /kaggle/tmp/model
!ls /kaggle/tmp
!pip install git+https://github.com/Maluuba/nlg-eval.git@master
!pip install wandb

In [ ]:
# !wandb login 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_tb = pd.read_csv('../input/indonesia-bible-tb/tb.csv')
df_vmd = pd.read_csv('../input/indonesia-bible-tb/vmd.csv')


In [ ]:
df_tb.head(5)
df_tb.rename({'firman': 'input_text'}, axis=1, inplace=True)
df_vmd.rename({'firman': 'target_text'}, axis=1, inplace=True)
df_tb.drop('id',axis=1,inplace=True)
df_vmd.drop('id',axis=1,inplace=True)


In [ ]:
df_tb.head(5)

In [ ]:
df_train = df_tb.merge(df_vmd,how='outer',left_on=['kitab','pasal','ayat'],right_on=['kitab','pasal','ayat'])

In [ ]:
df_tb['prefix'] = 'simplify'

In [ ]:
! pip uninstall torch torchvision -y
! pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U transformers
!pip install -U simpletransformers  

In [ ]:
train_df=df_train.sample(frac=0.5,random_state=200) #random state is a seed value
test_df=df_train.drop(train_df.index)


In [ ]:

df_train.drop(['kitab','pasal','ayat'],inplace=True,axis=1)

In [ ]:
df_train.head(5)

In [ ]:
!pip install torchsummary
from torchsummary import summary
# from summary import summary


In [ ]:
import logging

import transformers
from simpletransformers.t5 import T5Model

from simpletransformers.seq2seq import Seq2SeqModel
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



model_args = {
    "reprocess_input_data": True,
#     "no_save": True,
    "max_seq_length": 50,
    "train_batch_size": 100,
    "output_dir" : save_loc,
    "num_train_epochs": 10,
    "wandb_project" : "Bible Paraphrase",
    "save_model_every_epoch": False,
    "overwrite_output_dir" :True,
    "max_length": 50,
#     "do_sample" : True,
#     "num_beams" : None,
#     "num_return_sequences" : 3,
#     "top_k" : 50,
#     "top_p" : 0.95,
}

# Initialize model
# model = Seq2SeqModel(
#     encoder_decoder_type="marian",
#     encoder_decoder_name="sshleifer/student_marian_en_ro_6_4",
#     args=model_args,
# )
model = T5Model("t5-base",None, args=model_args)
# Train the model

# Evaluate the model
# results = model.eval_model(eval_df)

In [ ]:
train_df['prefix'] = 'paraphrase'

In [ ]:

model.train_model(train_df,use_cuda=True)

In [ ]:

# Use the model for prediction
print(model.predict(['Dan Allah menamai terang itu siang, dan gelap itu malam. Jadilah petang dan jadilah pagi, itulah hari pertama.']))


In [ ]:

test_df.reset_index(inplace=True)

In [ ]:
df_train.head(5)
def print_df(df,index):
    print(df.loc[index]['input_text'])
    print(df.loc[index]['target_text'])
    print(model.predict([df.loc[index]['input_text']]))
    

In [ ]:
print_df(test_df,6)

In [ ]:
from tqdm import tqdm
len(test_df)

In [ ]:
# results = model.eval_model(test_df,output_dir='./')
res = model.predict(test_df.loc[:5000]['input_text'].tolist())

In [ ]:
df_eval = test_df.head(5001)

In [ ]:
df_eval.head(5)
df_eval['pred'] = res

In [ ]:
df_eval.head(5)

In [ ]:
!du -h /kaggle/tmp/model

In [ ]:
df_eval.to_csv('result.csv',index=False)

In [ ]:
len(df_eval)

In [ ]:
!pip install sacrebleu

In [ ]:
import sacrebleu
refs = [df_eval['target_text'].tolist()]
sys = df_eval['pred'].tolist()
bleu = sacrebleu.corpus_bleu(sys, refs)
print(bleu.score)

In [ ]:
from nlgeval import NLGEval
nlgeval = NLGEval()  # loads the models
metrics_dict = nlgeval.compute_metrics(df_eval['target_text'], df_eval['pred'])

In [ ]:
len(sys)